For this kernel I descided not to repeat what others did, so there are no visualizations of the data, and class analysis, you can go to other notebooks for that. I descided that I will go straight to the point :)

So this will be our network:
<img src="https://imgur.com/C5YptiK.png">

In [ ]:
import numpy as np
import pandas as pd
import os
print(os.listdir("../input"))

### Loading the data

In [ ]:
x_train_data = pd.read_csv('../input/X_train.csv')
y_train_data = pd.read_csv('../input/y_train.csv')
x_test_data = pd.read_csv('../input/X_test.csv')

In [ ]:
x_train_data.head()

### Data preparation

In [ ]:
x_train_data.drop(['series_id', 'measurement_number'], axis=1).describe()

Lets add some features :)

As we see not all data columns have zero mean and variance of 1. To help the training process we going to normalize the data with standard scaller.

In [ ]:
def quaternion_to_euler(x, y, z, w):
    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.degrees(math.atan2(t0, t1))

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.degrees(math.asin(t2))

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.degrees(math.atan2(t3, t4))

    return X, Y, Z

In [ ]:
def feature_engineering(data):
    eulers = np.array([quaternion_to_euler(*x) for x in data[['orientation_X', 'orientation_Y', 'orientation_Z', 'orientation_W']].values])
    data['euler_orientation_x'] = eulers[:, 0]
    data['euler_orientation_y'] = eulers[:, 1]
    data['euler_orientation_z'] = eulers[:, 2]

    accCols = ['linear_acceleration_X', 'linear_acceleration_Y', 'linear_acceleration_Z']
    gyroCols = ['angular_velocity_X', 'angular_velocity_Y', 'angular_velocity_Z']
    quatCols = ['orientation_X', 'orientation_Y', 'orientation_Z', 'orientation_W']
    gyroEulerCols = ['euler_orientation_x', 'euler_orientation_y', 'euler_orientation_z']

    data['angular_velocity_norm'] = np.sqrt(np.sum(np.square(data[gyroCols]),axis=1))
    data['orientation_norm'] = np.sqrt(np.sum(np.square(data[quatCols]),axis=1))
    data['linear_acceleration_norm'] = np.sqrt(np.sum(np.square(data[accCols]),axis=1))
    data['euler_orientation_norm'] = np.sqrt(np.sum(np.square(data[gyroEulerCols]),axis=1))
    data['acc_vs_vel'] = data['linear_acceleration_norm'] / data['angular_velocity_norm']
    
    return data

x_train_data = feature_engineering(x_train_data)
x_test_data = feature_engineering(x_test_data)

In [17]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
original_series_id = x_train_data['series_id']
df_x_train = pd.DataFrame(sc.fit_transform(x_train_data.drop(['row_id', 'series_id', 'measurement_number'], axis=1)),
                       columns=x_train_data.drop(['row_id', 'series_id', 'measurement_number'], axis=1).columns)
df_x_train['series_id'] = original_series_id
df_x_train.describe()

,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,euler_orientation_x,euler_orientation_y,euler_orientation_z,angular_velocity_norm,orientation_norm,linear_acceleration_norm,euler_orientation_norm,acc_vs_vel,series_id
count,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,4.876800e+05,487680.000000
mean,-6.313893e-17,4.491886e-15,1.938412e-15,-2.961005e-16,-2.600537e-17,-3.140226e-16,-1.569629e-16,-1.333096e-17,1.054999e-16,-9.197266e-16,-5.275495e-15,-2.077760e-15,-1.300913e-16,-3.532297e-16,1.028696e-11,6.801249e-16,3.774655e-15,-4.406076e-16,1904.500000
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1099.853353
min,-1.416154e+00,-1.503351e+00,-1.654092e+00,-1.465171e+00,-2.013508e+01,-1.055745e+01,-5.453192e+00,-1.935011e+01,-5.811808e+01,-2.320326e+01,-5.064095e+00,-4.670922e+00,-1.588886e+00,-1.065295e+00,-2.674867e+00,-3.962085e+00,-1.298703e+00,-3.950798e-01,0.000000
25%,-1.002005e+00,-1.078812e+00,-9.617994e-01,-9.804119e-01,-3.475569e-01,-4.683154e-01,-3.122764e-01,-3.528890e-01,-4.338970e-01,-2.910425e-01,-3.680604e-01,-5.908340e-01,-8.179249e-01,-6.621412e-01,-7.837094e-01,-3.973346e-01,-8.922462e-01,-2.879285e-01,952.000000
50%,-1.282060e-01,2.298604e-01,1.839337e-01,-1.428654e-01,-7.924474e-04,-3.299045e-02,6.043561e-02,-2.299244e-03,-3.209117e-03,-1.455907e-04,-1.190265e-01,7.473167e-03,-2.885605e-01,-3.409779e-01,6.687445e-04,-1.505976e-01,-1.809789e-01,-1.940077e-01,1904.500000
75%,9.768948e-01,1.037082e+00,1.041900e+00,9.685492e-01,3.426337e-01,4.480377e-01,3.656402e-01,3.544222e-01,4.263105e-01,2.959879e-01,1.144805e-01,6.676946e-01,9.443379e-01,2.595756e-01,7.802151e-01,2.270091e-01,7.763260e-01,-3.418381e-02,2857.000000
max,1.468801e+00,1.290433e+00,1.351793e+00,1.520372e+00,1.937801e+01,1.207493e+01,6.137744e+00,1.960214e+01,3.276607e+01,2.643056e+01,5.914152e+00,6.313439e+00,1.897833e+00,1.203852e+01,2.617213e+00,5.227411e+01,2.216526e+00,7.235175e+01,3809.000000


We need to use the same scaller parameters for the test data.

In [18]:
original_series_id = x_test_data['series_id']
df_x_test = pd.DataFrame(sc.transform(x_test_data.drop(['row_id', 'series_id', 'measurement_number'], axis=1)),
                       columns=x_test_data.drop(['row_id', 'series_id', 'measurement_number'], axis=1).columns)
df_x_test['series_id'] = original_series_id

In [19]:
series_id = x_train_data.series_id.unique()
set([len(df_x_train[df_x_train['series_id'] == row_id]) for row_id in series_id])

{128}

We need to get as much data as we can, so we add sliding window.

In [20]:
def get_sliding_window(x, y, seq_length, shift):
    data_grouped_by_classes = {key: [] for key in set(y['surface'].values)}
    for class_name in set(y['surface'].values):
        for y_row in y.iterrows():
            y_row = y_row[1]
            if y_row['surface'] == class_name:
                data_grouped_by_classes[class_name].append([x[x['series_id'] == y_row['series_id']], y_row['surface']])       
    
    x_ret = []
    y_ret = []
    for class_name in data_grouped_by_classes.keys():
        for arr in data_grouped_by_classes[class_name]:
            arr_y = arr[1]
            arr_x = arr[0].values
            batch_size_total = seq_length
            n_batches = len(arr_x)//batch_size_total
            arr_x = arr_x[:n_batches * batch_size_total]
            for n in range(0, arr_x.shape[0], shift):  
                if n + seq_length > arr_x.shape[0]:
                    break
                x_batch = arr_x[n:n+seq_length]
                y_batch = arr_y
                x_ret.append(x_batch)
                y_ret.append(y_batch)
    return x_ret, y_ret

def get_data(x, y, batch_size, seq_length, shift):
    import random
    data_x = []
    data_y = []
    for x_, y_ in zip(*get_sliding_window(x, y, seq_length, shift)):
        data_x.append(x_)
        data_y.append(y_)
    
    c = list(zip(data_x, data_y))
    random.shuffle(c)
    data_x, data_y = zip(*c)
    ret_x = []
    ret_y = []
    
    for i in range(0, len(data_x), batch_size):
        if len(data_x) <= i+batch_size:
            break
        ret_x.append(np.array(data_x[i:i+batch_size])[0])
        ret_y.append(np.array(data_y[i:i+batch_size])[0])
    return np.array(ret_x), np.array(ret_y)

In [21]:
x_train, y_train = get_data(df_x_train, y_train_data, 1, 16, 2)
x_train = x_train[:, :, :-1]
x_train = x_train.reshape(-1, 4, 4, 18)

In [22]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

enc_l = LabelEncoder()
enc = OneHotEncoder(handle_unknown='ignore')
nr_y_train = enc_l.fit_transform(np.array(y_train))
one_hot_y_train = enc.fit_transform(nr_y_train.reshape(-1, 1))

### Deep Learning Model

In [23]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM, GRU, SimpleRNN, Conv1D, TimeDistributed, MaxPooling1D, Flatten, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

In [24]:
def init_model():
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=8, kernel_size=2, activation='relu', padding='same'), batch_input_shape=(None, None, 4, 18)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(16))
    model.add(Dense(9, activation='softmax'))
    
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)

    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

Time for training

In [25]:
model = init_model()
callbacks = [EarlyStopping('val_loss', patience=3)]
model.fit(x_train, one_hot_y_train, epochs=10, batch_size=128, validation_split=0.2, callbacks=callbacks)

Train on 173735 samples, validate on 43434 samples
Epoch 1/10
173735/173735 [==============================] - 15s 85us/sample - loss: 1.2104 - acc: 0.5703 - val_loss: 0.8577 - val_acc: 0.7044
Epoch 2/10
173735/173735 [==============================] - 13s 76us/sample - loss: 0.7153 - acc: 0.7535 - val_loss: 0.6142 - val_acc: 0.7880
Epoch 3/10
173735/173735 [==============================] - 13s 76us/sample - loss: 0.5622 - acc: 0.8082 - val_loss: 0.5185 - val_acc: 0.8188
Epoch 4/10
173735/173735 [==============================] - 13s 78us/sample - loss: 0.4858 - acc: 0.8313 - val_loss: 0.4606 - val_acc: 0.8388
Epoch 5/10
173735/173735 [==============================] - 13s 77us/sample - loss: 0.4343 - acc: 0.8484 - val_loss: 0.4131 - val_acc: 0.8558
Epoch 6/10
173735/173735 [==============================] - 13s 77us/sample - loss: 0.3967 - acc: 0.8600 - val_loss: 0.3801 - val_acc: 0.8670
Epoch 7/10
173735/173735 [==============================] - 16s 89us/sample - loss: 0.3682 - acc:

In [27]:
series_id = df_x_test.series_id.unique()
our_predictions = []
for row_id in series_id:
    batch_data = df_x_test[df_x_test['series_id'] == row_id].drop(['series_id'], axis=1).values[:128, :].reshape(8, 16, 18).reshape(8, 4, 4, 18)
    predictions = model.predict(batch_data)
    final_predictions = list(sum(predictions)/len(predictions))
    our_predictions.append(final_predictions)

label_rez = enc_l.inverse_transform(enc.inverse_transform(our_predictions))

df = pd.DataFrame({'series_id': series_id.tolist(),'surface': label_rez.tolist()})
df.to_csv('submission.csv', index=False)

KeyboardInterrupt: 